# Models

This notebook will demonstrate the basic features of AutoMPC for system ID modeling and model evaluation.

## Set-Up

As before, we begin by importing autompc.

In [1]:
import autompc as ampc
import numpy as np

Loading AutoMPC...
Finished loading AutoMPC


To perform system identification, we need a dataset of trajectories to work with.  We will use the cartpole system, available from the `benchmarks` package, to generate our dataset.

In [2]:
from autompc.benchmarks.mcube_pushing import MCubePushingBenchmark

benchmark = MCubePushingBenchmark()

system = benchmark.system
trajs = benchmark.get_trajs(num_trajs=100)

Let's import some useful utilities for debugging, as well.  Here, we'll plot one of the training trajectories and some statistics about the overall training set.

In [3]:
%matplotlib inline
from autompc.sysid.metrics import get_model_rmse
from autompc.graphs import plot_traj,plot_traj_projected,plot_simulation,plot_rollout,plot_trajs
import matplotlib.pyplot as plt

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(14,5))

plot_traj(trajs[0],ax=ax1)
#plot_traj_projected(trajs[0],ax=ax1)
ax1.set_ylim(-10,10)
plot_trajs(trajs,style='confidence',ax=ax2)
ax2.set_ylim(-10,10)
plt.plot()

ImportError: cannot import name 'ModelTuneResult' from 'autompc.tuning.model_tuner' (/home/william/proj/autompc_0.2-dev/autompc/tuning/model_tuner.py)

To double-check that the dynamics are working, let's simulate a zero-control trajectory from 45 degrees tilt.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from autompc import Trajectory
from autompc.utils import rollout
from autompc.benchmarks.cartpole import cartpole_simp_dynamics
from autompc.graphs import plot_traj,plot_traj_projected,plot_simulation,plot_rollout,plot_trajs

traj = Trajectory.zeros(system,100)
traj.obs[0,0] = np.pi/16
traj.ctrls[:] = 10

plot_rollout(traj,benchmark.dynamics,compare=False)
plt.ylim(-6,6)
plt.show()

## Models

AutoMPC provides a variety of sytem ID models which can be used to learn the sytem dynamics.  Here, we will use an Multi-Layer Perceptron (MLP) neural network model, but for a complete list see [here](https://autompc.readthedocs.io/en/latest/source/sysid.html#supported-system-id-models).

You can examine the hyperparameters used for training using `get_config()` and change them using `set_hyper_values()`, as shown below.  Here we'll change batch normalization to false and the number of hidden layers in the neural net.

(**Note:** This will take several minutes to run depending on your hardware).

In [ ]:
from autompc.sysid import MLP

model = MLP(system,n_train_iters=50)
print("Original hyperparameter configuration")
print(model.get_config())                    #prints the hyperparameters of the model
model.set_hyper_values(batchnorm=False)      #modifies the hyperparameters
model.set_hyper_values(n_hidden_layers='3')  #modifies the hyperparameters
print("Modified hyperparameter configuration")
print(model.get_config())                    #prints the hyperparameters of the model

assert model.is_trained == False
model.train(trajs)                           #performs training
assert model.is_trained == True

When manually training models like this, it is a good idea to inspect your training and testing error. You can do so with some convenience methods in AutoMPC.  The below code will compute the Root-Mean-Squared Error (RMSE) and also plot a rollout of the model, using the initial state and controls from a sample trajectory

In [ ]:
print("MLP 1-step RMSE on training set:",get_model_rmse(model,trajs))
print("MLP 1-step RMSE on testing set:",get_model_rmse(model,test_trajs))

plot_rollout(trajs[0],model)
plt.ylim([-20,20])
plt.show()

Now that we have trained our model, we can use it to make predictions.  Let's try predicting the next state from one of our training trajectories.  We first compute the model state at a certain point in the trajectory

In [ ]:
traj = trajs[0]
model_state = model.traj_to_state(traj[:100])

The model state contains the information the model needs to predict the next time step.  `model_state[:system.obs_dim]` is always equal to the most recent observation.  For the MLP, that's actually all there is to the model state, but some models require a larger state.  We can see the dimension of the model state by running

In [ ]:
model.state_dim

We can also check other properties of the model, such as whether it is differentiable and whether it is linear.

In [ ]:
print("Model is Differentiable? ", model.is_diff)
print("Model is Linear? ", model.is_linear)

For comparison, consider the ARX model.  

In [ ]:
from autompc.sysid import ARX

model_arx = ARX(system)
model_arx.train(trajs)

print("ARX 1-step RMSE on training set:",get_model_rmse(model_arx,trajs))
print("ARX 1-step RMSE on testing set:",get_model_rmse(model_arx,test_trajs))

plot_rollout(trajs[0],model_arx)
plt.ylim([-20,20])
plt.show()

We observe that, unlike the MLP model, the ARX model state size is larger than `system.obs_dim` since the model state includes the history of several observations.  Make sure to use the `traj_to_state` method to properly derive the model state.  We can also observe that the ARX model is linear, which means that it is suitable for use with LQR control.

In [ ]:
model_state_arx = model_arx.traj_to_state(traj[:100])
model_arx.state_dim

In [ ]:
model_arx.is_linear

We can use our current model state, and the control to make a prediction of the new model state

In [ ]:
pred_state = model.pred(model_state, traj[99].ctrl)
pred_state

Compare this to the true observation

In [ ]:
traj[100].obs

We can use the true observation to update our model state

In [ ]:
new_model_state = model.update_state(model_state, traj[99].ctrl, traj[100].obs)
new_model_state

For differentiable models, we can also get the Jacobian of the
model prediction

In [ ]:
pred_state, state_jac, ctrl_jac = model.pred_diff(model_state, traj[99].ctrl)
state_jac

## Graphing Model Accuracy

Now, we'd like to compare ARX to our original model.  One convenient way to do this is by graphing the model prediction error over various prediction horizons.  AutoMPC provides tools for easily constructing this graph.  (**Note:** This may take a few minutes to run)

In [ ]:
import matplotlib.pyplot as plt
from autompc.graphs import KstepPredAccGraph

graph = KstepPredAccGraph(system, trajs, kmax=20, metric="rmse")
graph.add_model(model, "MLP")
graph.add_model(model_arx, "ARX")

fig = plt.figure()
ax = fig.gca()
graph(fig, ax)
ax.set_title("Comparison of models")
plt.show()

We can see that the MLP has much lower prediction error than the ARX, and that the prediction error of both models increases with the horizon.  Here the system timestep is 0.05s, so the end of the plot is about 1 second in the future.

Another thing that we often like to do is to compare the rollouts of two methods. We can do so using the `plot_traj` method multiple times on the trajectory prediction error, as shown below. It can be seen that ARX does a better job in very long-term prediction particularly on the x and dx observations, whereas MLP does a better job on short-range prediction.

In [ ]:
#compute rollout errors
from autompc.utils import rollout

index = 3
#dims = ['x']
dims = None
traj_mlp = rollout(model,trajs[index])
traj_arx = rollout(model_arx,trajs[index])
mlp_err = traj_mlp-trajs[index]
arx_err = traj_arx-trajs[index]
plot_traj(mlp_err,obs_opts=[{'label':'MLP '+x} for x in traj_mlp.system.observations],ctrl_opts=False,dims=dims)
plot_traj(arx_err,obs_opts=[{'label':'ARX '+x,'linestyle':'--'} for x in traj_arx.system.observations],ctrl_opts=False,dims=dims)
plt.title("Rollout errors")
plt.plot()

In [ ]:
#compute data set of errors and plot stats for both models
errs_mlp = [None]*len(trajs)
errs_arx = [None]*len(trajs)
for i in range(len(trajs)):
    errs_mlp[i] = rollout(model,trajs[i])-trajs[i]
    errs_arx[i] = rollout(model_arx,trajs[i])-trajs[i]

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(14,5))

plot_trajs(errs_mlp,style='confidence',ctrl_opts=False,ax=ax1)
plot_trajs(errs_arx,style='confidence',ctrl_opts=False,ax=ax2)
ax1.set_title("MLP errors")
ax2.set_title("ARX errors")
plt.plot()